# DTLZ2 Benchmark
## Imports

In [1]:
from bofire.benchmarks.multi import DTLZ2, C2DTLZ2
from bofire.utils.multiobjective import compute_hypervolume
from bofire.data_models.strategies.api import QehviStrategy, QparegoStrategy, RandomStrategy, PolytopeSampler, SoboStrategy
import bofire.strategies.api as strategies
from bofire.data_models.api import Domain, Outputs, Inputs
from bofire.data_models.features.api import ContinuousInput, ContinuousOutput, CategoricalOutput, CategoricalInput
from bofire.data_models.objectives.api import MinimizeObjective, MinimizeSigmoidObjective, MaximizeSigmoidObjective
from functools import partial
import pandas as pd
import os
from bofire.plot.api import plot_objective_plotly



c:\ProgramData\Anaconda3\envs\bofire-env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Manual setup of the optimization domain

The following cell shows how to manually setup the optimization problem in BoFire for didactic purposes. In the following the implemented benchmark module is then used.

In [2]:
input_features = Inputs(features=[ContinuousInput(key=f"x_{i}", bounds=(0, 1)) for i in range(5)] + [CategoricalInput(key=f"x_5", categories=(0.5, 0.0))])
# here the minimize objective is used, if you want to maximize you have to use the maximize objective.
output_features = Outputs(features=[
        ContinuousOutput(key=f"f_{0}", objective=MinimizeObjective(w=1.)),
        CategoricalOutput(key=f"f_{1}", categories=["infeasible", "feasible"], objective=[0, 1])
        # ContinuousOutput(key=f"f_{1}", objective=MinimizeSigmoidObjective(w=1., steepness=50, tp=0.25)),
    ]
)
# no constraints are present so we can create the domain
domain1 = Domain(inputs=input_features, outputs=output_features)

# plot_objective_plotly(domain.outputs.get_by_key("f_0"), lower=0, upper=2)

In [3]:
import numpy as np
sample_df = domain1.inputs.sample(20).astype(float) # Sample x's

# Write a function which outputs one continuous variable and another discrete based on some logic
sample_df["f_0"] = np.cos(sample_df.values.sum(1))
sample_df["f_1"] = "infeasible"
sample_df["f_1"][sample_df["x_0"]+sample_df["x_1"] <= 1.0] = "feasible"
sample_df[
            [
                "valid_%s" % feat
                for feat in domain1.outputs.get_keys_by_objective(  # type: ignore
                    includes=[MinimizeObjective]
                )
            ]
        ] = 1

sample_df

C:\Users\G15361\AppData\Local\Temp\ipykernel_25932\2798685259.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_df["f_1"][sample_df["x_0"]+sample_df["x_1"] <= 1.0] = "feasible"


,x_0,x_1,x_2,x_3,x_4,x_5,f_0,f_1,valid_f_0
0,0.769508,0.240069,0.277084,0.736428,0.344370,0.5,-0.962660,infeasible,1
1,0.788241,0.777982,0.892712,0.542482,0.109289,0.0,-0.999523,infeasible,1
2,0.299695,0.544068,0.546406,0.169619,0.000094,0.0,0.010914,feasible,1
3,0.217495,0.753068,0.836795,0.337995,0.683057,0.0,-0.951358,feasible,1
4,0.626194,0.779950,0.077964,0.181630,0.728884,0.0,-0.733750,infeasible,1
5,0.200626,0.169175,0.085977,0.931861,0.892676,0.0,-0.651469,feasible,1
6,0.122709,0.527760,0.818104,0.076293,0.535340,0.0,-0.487663,feasible,1
7,0.309827,0.617472,0.080858,0.258398,0.628364,0.5,-0.733951,feasible,1
8,0.457409,0.800157,0.568710,0.765764,0.416211,0.0,-0.991123,infeasible,1
9,0.834006,0.392399,0.148989,0.433485,0.104670,0.5,-0.746477,infeasible,1


## Setup of the Strategy and ask for Candidates



In [4]:
from bofire.data_models.acquisition_functions.api import qNEI, qUCB, qSR, qEI
from bofire.data_models.strategies.api import QparegoStrategy, MultiplicativeSoboStrategy, SoboStrategy
from bofire.data_models.surrogates.api import BotorchSurrogates, MLPEnsemble
from bofire.data_models.domain.api import Outputs

strategy_data = SoboStrategy(domain=domain1, 
                             acquisition_function=qEI(), 
                             surrogate_specs=BotorchSurrogates(surrogates=
                                    [
                                        MLPEnsemble(inputs=domain1.inputs, outputs=Outputs(features=[domain1.outputs[1]]), lr=1.0, n_epochs=100)
                                    ]
                                )
                            )

strategy = strategies.map(strategy_data)

# experiments = DTLZ2(dim=6).f(domain1.inputs.sample(20).astype(float), return_complete=True)

strategy.tell(sample_df)
candidates = strategy.ask(10)

print(candidates)

X:          x_0       x_1       x_2       x_3       x_4  x_5
0   0.769508  0.240069  0.277084  0.736428  0.344370  0.5
1   0.788241  0.777982  0.892712  0.542482  0.109289  0.0
2   0.299695  0.544068  0.546406  0.169619  0.000094  0.0
3   0.217495  0.753068  0.836795  0.337995  0.683057  0.0
4   0.626194  0.779950  0.077964  0.181630  0.728884  0.0
5   0.200626  0.169175  0.085977  0.931861  0.892676  0.0
6   0.122709  0.527760  0.818104  0.076293  0.535340  0.0
7   0.309827  0.617472  0.080858  0.258398  0.628364  0.5
8   0.457409  0.800157  0.568710  0.765764  0.416211  0.0
9   0.834006  0.392399  0.148989  0.433485  0.104670  0.5
10  0.058660  0.703493  0.731653  0.114617  0.867340  0.0
11  0.174764  0.141889  0.393035  0.749079  0.770134  0.0
12  0.289699  0.463533  0.319879  0.019126  0.523205  0.0
13  0.446521  0.543391  0.628782  0.274490  0.172218  0.5
14  0.383332  0.534124  0.083969  0.928272  0.362520  0.0
15  0.088455  0.671932  0.299365  0.262484  0.723244  0.5
16  0.30717

c:\ProgramData\Anaconda3\envs\bofire-env\lib\site-packages\botorch\models\model.py:214: RuntimeWarning: Could not update `train_inputs` with transformed inputs since _MLPClassifierEnsemble does not have a `train_inputs` attribute. Make sure that the `input_transform` is applied to both the train inputs and test inputs.
  warnings.warn(


        x_0       x_1       x_2       x_3       x_4  x_5  f_0_pred   f_1_pred  \
0  0.326234  1.000000  1.000000  1.000000  1.000000  0.5 -2.118929 -14.356292   
1  1.000000  1.000000  1.000000  1.000000  1.000000  0.5 -1.965952  -1.856444   
2  0.599940  1.000000  1.000000  0.391617  1.000000  0.5 -1.997947 -13.875225   
3  1.000000  1.000000  1.000000  0.000000  1.000000  0.5 -1.829928  -8.801380   
4  1.000000  1.000000  0.000000  1.000000  1.000000  0.0 -1.794281   0.282144   
5  0.000000  1.000000  1.000000  1.000000  1.000000  0.5 -2.068119 -26.166718   
6  1.000000  1.000000  1.000000  1.000000  1.000000  0.0 -1.924939   0.120979   
7  0.846747  0.983599  0.551493  0.331172  1.000000  0.0 -1.546147  -0.139778   
8  0.835615  0.250119  0.238424  0.642778  0.343853  0.5 -0.929888   0.251818   
9  0.716120  0.923542  1.000000  0.301291  0.822325  0.5 -1.791427 -11.423274   

     f_0_sd     f_1_sd   f_0_des  f_1_des  
0  0.196473  18.588478  2.118929      NaN  
1  0.239787   3.6876

c:\ProgramData\Anaconda3\envs\bofire-env\lib\site-packages\botorch\models\model.py:214: RuntimeWarning: Could not update `train_inputs` with transformed inputs since _MLPClassifierEnsemble does not have a `train_inputs` attribute. Make sure that the `input_transform` is applied to both the train inputs and test inputs.
  warnings.warn(
c:\ProgramData\Anaconda3\envs\bofire-env\lib\site-packages\botorch\models\model.py:214: RuntimeWarning: Could not update `train_inputs` with transformed inputs since _MLPClassifierEnsemble does not have a `train_inputs` attribute. Make sure that the `input_transform` is applied to both the train inputs and test inputs.
  warnings.warn(


In [5]:
from bofire.data_models.objectives.api import Objective
strategy.domain.outputs.get_by_objective(includes=[Objective, list])

Outputs(type='Outputs', features=[ContinuousOutput(type='ContinuousOutput', key='f_0', unit=None, objective=MinimizeObjective(type='MinimizeObjective', w=1.0, bounds=(0, 1))), CategoricalOutput(type='CategoricalOutput', key='f_1', categories=('infeasible', 'feasible'), objective=[0.0, 1.0])])

In [6]:
print(domain1.outputs)

print(strategy._get_objective_and_constraints())

print(strategy.acquisition_function)

type='Outputs' features=[ContinuousOutput(type='ContinuousOutput', key='f_0', unit=None, objective=MinimizeObjective(type='MinimizeObjective', w=1.0, bounds=(0, 1))), CategoricalOutput(type='CategoricalOutput', key='f_1', categories=('infeasible', 'feasible'), objective=[0.0, 1.0])]
(GenericMCObjective(), None, 0.001)
type='qEI'


## Test gpytorch things

In [7]:
import torch
from gpytorch.likelihoods import BernoulliLikelihood

# Set up dummy tensors for easy verification
x = torch.tensor([[1.0, 2.0], [3.0, 4.0]])
y = torch.tensor([0.5, 0.0], requires_grad=True)
out = torch.matmul(x, y)
print(f"=== Original ===")
print(f"Should be X^T 1 for 1 the all ones vector")
print(out)
out = out.sum()
out.backward()
print(y.grad)

print(f"\n=== Likelihood ===")
like = BernoulliLikelihood()
x1 = torch.tensor([[1.0, 2.0], [3.0, 4.0]])
y1 = torch.tensor([0.5, 0.0], requires_grad=True)
out1 = like(torch.matmul(x1, y1))
print(out1.probs)
out1 = out1.probs.sum()
print(out1)
out1.backward()
print(y1.grad)

=== Original ===
Should be X^T 1 for 1 the all ones vector
tensor([0.5000, 1.5000], grad_fn=<MvBackward0>)
tensor([4., 6.])

=== Likelihood ===
tensor([0.6915, 0.9332], grad_fn=<MulBackward0>)
tensor(1.6247, grad_fn=<SumBackward0>)
tensor([0.7406, 1.2222])


# Add Classification Models for Surrogates

Updating the surrogates to allow for classification of output values (i.e. 'feasible' or 'infeasible').

### Housekeeping changes

1. Update the categorical input/outputs ('bofire/data_models/features/categorical.py') to always return a tuple instead of a list for `categories` and attribute (to prevent mutation)
    - Associated test are changed in 'tests/bofire/data_models/specs/features.py'
2. 

### Classification Models

Initially, we are only interested in checking whether or not certain points are feasible or infeasible, hence this is a binary classification problem. 


### Questions

1. Should we force `allowed` to be a tuple for the categorical input/outputs? If so, we need to refactor indexing for Pandas DFs...